# SigLIP

Revisión informal de _SigLIP_ (_Sigmoid Loss for Language Image Pre-Training_).

Modelo de código abierto publicado en marzo de 2023.

- [https://arxiv.org/pdf/2303.15343](https://arxiv.org/pdf/2303.15343)

Aunque los términos arquitectura y modelo se utilicen indistintamente, es habitual presentar _SigLIP_ como un método de _pre-entrenamiento_.

## 1. Arquitectura

_SigLIP_ es una variante de la arquitectura _CLIP_ (_Contrastive Language-Image Pre-Training_).

- [https://github.com/jcmellado/sapiens/blob/main/notebooks/pre-training/clip.ipynb](https://github.com/jcmellado/sapiens/blob/main/notebooks/pre-training/clip.ipynb)

Sigue la misma arquitectura, pero utiliza un mecanismo distinto para entrenar el modelo.

### 1.1. Motivación

_CLIP_ procesa _batches_ de $N$ imágenes y $N$ textos, donde la imagen en la posición $i$ se corresponde con el texto en la posición $i$.

Para cada _batch_ genera puntuaciones con las que construye una matriz de $N \times N$ dimensiones, buscando maximizar la puntuación de los elementos $(i, j)$ de la diagonal ($j = i$), y minimizar las puntuaciones del resto de elementos fuera de la diagonal ($j \ne i$).

Pero asume que para una imagen $i$, los textos en las posiciones $(i, j)$ con $j \ne i$ no están relacionados de ninguna forma con la imagen $i$, y viceversa, lo cual no tiene por qué ser cierto.

_SigLIP_ propone un método de entrenamiento alternativo para reducir el impacto de esta asunción, y optimizar además los cálculos a realizar.

## 2. Funciones de Pérdida

El cambio propuesto por el modelo se fundamenta en el funcionamiento de distintas funciones de pérdidas, por lo que el análisis se enfoca desde esa perspectiva.

### 2.1. Cross Entropy Loss

_CLIP_ utiliza la función de pérdida de entropía cruzada (_Cross Entropy Loss_).

Lo que puede expresarse utilizando la formulación del _paper_ original:

- $ - \dfrac{1}{2|\mathcal{B}|} \sum\limits_{i=1}^{|\mathcal{B}|} \left( \log \overbrace{ \dfrac{e^{t \mathbf{x}_i \cdot \mathbf{y}_i}}{\sum\limits_{j=1}^{|\mathcal{B}|} e^{t \mathbf{x}_i \cdot \mathbf{y}_j}}}^{\text{image vs text softmax}} + \log \overbrace{ \dfrac{e^{t \mathbf{x}_i \cdot \mathbf{y}_i}}{\sum\limits_{j=1}^{|\mathcal{B}|} e^{t \mathbf{x}_j \cdot \mathbf{y}_i}}}^{\text{text vs image softmax}} \right) $

Siendo:

- $\mathbf{x}$ e $\mathbf{y}$: los vectores de características normalizados, para imágenes y textos respectivamente, obtenidos por el modelo.

- $t$ la: _temperatura_, un factor de escala calculado como $t = e^{t'}$, siendo $t'$ un parámetro global aprendido por el modelo.

El modelo calcula la _similitud coseno_ como el producto escalar de los vectores de características normalizados $\mathbf{x}$ e $\mathbf{y}$, y la escala con la temperatura $t$.

Aplica $softmax$ dos veces, una vez considerando la dimensión de las imágenes contra los textos, y otra vez considerando la dimensión de los textos contra las imágenes.

- $ \operatorname{softmax}(x_i)_j = \dfrac{e^{x_{ij}}}{\sum\limits_{k=1}^{C} e^{x_{ik}}} $

Calcula el logaritmo con signo negativo para penalizar los errores, a menor probabilidad, mayor error.

Utiliza etiquetas implícitas, positivas de valor $1$ para los elementos sobre la diagonal, y $0$ para el resto.

Y calcula la pérdida total como la media de los dos resultados obtenidos.

_CLIP_ considera todas las puntuaciones a lo largo de cada dimensión para calcular las probabilidades, lo que implica calcular dos factores globales de normalización (los sumatorios de los divisores), lo que es computacionalmente ineficiente y aumenta las necesidades de memoria.

### 2.2. Sigmoid Loss

_SigLIP_ introduce el uso de la función de pérdida _Sigmoid Loss_ basada en la función sigmoide.

Lo que puede expresarse utilizando la formulación del _paper_ original:

- $ - \dfrac{1}{|\mathcal{B}|} \sum\limits_{i=1}^{|\mathcal{B}|} \sum\limits_{j=1}^{|\mathcal{B}|} \underbrace{\log \dfrac{1}{1 + e^{z_{ij} (-t \mathbf{x}_i \cdot \mathbf{y}_j  + b)}}}_{\mathcal{L}_{ij}} $

Siendo:

- $\mathbf{x}$ y $\mathbf{y}$: los vectores de características normalizados, para imágenes y textos respectivamente, obtenidos por el modelo.

- $z$: las etiquetas, con valor $1$ para las clases correctas, y valor $-1$ para las incorrectas.

- $t$: la _temperatura_, un factor de escala calculado como $t = e^{t'}$, siendo $t'$ un parámetro global aprendido por el modelo, inicializado a $ln(10)$ en el _paper_.

- $b$: un _bias_, un parámetro global aprendido por el modelo, añadido para evitar la proliferación de valores negativos al empezar el entrenamiento, inicializado a $-10$ en el _paper_.

El modelo calcula la _similitud coseno_ como el producto escalar de los vectores de características normalizados $\mathbf{x}$ e $\mathbf{y}$, la escala con la temperatura $t$, y le suma el _bias_ $b$.

Utiliza etiquetas $z_{ij}$ explícitas, positivas de valor $1$ para los elementos sobre la diagonal, y valor negativo de $-1$ para el resto.

Aplica la función sigmoide a los valores calculados sobre la matriz de puntuaciones, elemento a elemento.

- $ \operatorname{sigmoid}(x_{ij}) = \dfrac{1}{1 + e^{-x_{ij}}} $

Calcula el logaritmo con signo negativo para penalizar los errores, a menor probabilidad, mayor error.

Y calcula la pérdida total como la suma de los resultados individuales obtenidos normalizando por el tamaño del _batch_.

_SigLIP_ considera sólo las puntuaciones individuales para calcular las probabilidades de cada clase, lo que evita el cálculo de los factores globales de normalización (los sumatorios de los divisores) presentes en el cálculo de la pérdida de _CLIP_.

### 2.3. Binary Cross Entropy

_CLIP_ considera el entrenamiento una tarea de clasificación con múltiples clases.

_SigLIP_ lo considera como una clasificación binaria.

Para entender esto es necesario introducir la expresión general de la función de pérdida de la entropía cruzada binaria (_Binary Cross Entropy_).

- $ - [z \ln(p) + (1 - z) \ln(1 - p)] $

Siendo:

- $z$ una etiqueta, con valor $1$ para la clase correcta, y $0$ para la incorrecta.

- $p$ la probabilidad predicha por el modelo para la etiqueta $z = 1$.

Para $z = 1$ la expresión general se simplifica para considerar sólo la probabilidad predicha por el modelo para la clase correcta.

- $ - \ln(p) $

Y para $z = 0$ se simplifica al contrario de la probabilidad predicha para la clase correcta.

- $ - \ln(1 - p) $

Por su parte, considerando la expresión general en _SigLIP_.

- $ -\ln(\operatorname{sigmoid}(z y))$.

Siendo:

- $z$: una etiqueta, con valor $1$ para la clase correcta, y $-1$ para la incorrecta.

- $y$: la puntuación bruta (_logit_) calculada por el modelo para la etiqueta $z$.

La probabilidad calculada por el modelo es el resultado de aplicar la función sigmoide al _logit_.

- $ p = \operatorname{sigmoid}(y) $

Para $z=1$ la expresión general se simplifica igual que en la función de pérdida de la entropía cruzada binaria.

- $ - \ln(\operatorname{sigmoid}(y)) = - \ln(p) $.

Y para $z=-1$ se simplifica de igual forma, aunque requiere desarrollar la expresión para demostrarlo.

- $ - \ln(\operatorname{sigmoid}(-y)) = - \ln(1 - \operatorname{sigmoid}(y)) = - \ln(1 - p) $.

Demostración:

- $ \operatorname{sigmoid}(-y) = \dfrac{1}{1 + e^y} $

- $ 1 - \operatorname{sigmoid}(y) = 1 - \dfrac{1}{1 + e^{-y}} = \dfrac{1 + e^{-y} - 1}{1 + e^{-y}} = \dfrac{e^{-y}}{1 + e^{-y}} \dfrac{e^y}{e^y} = \dfrac{1}{1 + e^y} $

## 3. Entrenamiento

### 3.1. Implementación

En el _paper_ se proporciona el siguiente _pseudocódigo_:

```python
# img_emb : image model embedding [n, dim]
# txt_emb : text model embedding [n, dim]
# t_prime, b : learnable temperature and bias
# n : mini-batch size

t = exp(t_prime)
zimg = l2_normalize(img_emb)
ztxt = l2_normalize(txt_emb)

logits = dot(zimg, ztxt.T) * t + b

labels = 2 * eye(n) - ones(n) # -1 with diagonal 1

l = -sum(log_sigmoid(labels * logits)) / n
```

Lo que corresponde a la siguiente secuencia de pasos:

- Calcular la _temperatura_ y normalizar los _embeddings_.

- Calcular los _logits_ como la _similitud coseno_, escalar por la _temperatura_, y sumar el _bias_.

- Construir las etiquetas explícitas como una matriz con valores $1$ en su diagonal, y $-1$ fuera de ella.

- Calcular la pérdida como la media de la función _log sigmoid_ aplicada al producto de las etiquetas por los _logits_.

  - $ \operatorname{log\_sigmoid}(x) = \ln(\operatorname{sigmoid}(x)) = \ln\left(\cfrac{1}{1 + e^{-x}}\right) = - \ln(1 + e^{-x}) $

Algunas líneas relevantes del código fuente:

```python
logits = jnp.dot(zimg, ztxt.T)
logits = logits * extras["t"] + extras["b"]

eye = jnp.eye(zimg.shape[0])
m1_diag1 = -jnp.ones_like(logits) + 2 * eye

loglik = jax.nn.log_sigmoid(m1_diag1 * logits)
nll = -jnp.sum(loglik, axis=-1)
l = jnp.mean(nll)
```

Notar que parece haber cierta discrepancia en el signo al calcular los _logits_ con respecto a la fórmula del _paper_.

### 3.2. Optimización

El _paper_ propone una técnica para calcular de forma eficiente la pérdida dividiendo el trabajo a realizar entre distintos dispositivos (_GPUs_ o _TPUs_).

El _batch_ de $N$ imágenes y $N$ textos de entrada se divide en fragmentos (_chunks_), y cada fragmento se envía a un dispositivo distinto.

Cada dispositivo calcula en paralelo y de forma independiente los vectores de características (_embeddings_) para las imágenes y textos de su fragmento, y la pérdida correspondiente.

Al terminar el cálculo de los _embeddings_ y la pérdida local inicial, cada dispositivo intercambia sus _embeddings_ de texto con otro dispositivo. Los de imágenes permanecen en el dispositivo, no se intercambian.

Cada dispositivo calcula entonces la pérdida con los nuevos _embeddings_ de texto recibidos, acumulando con la pérdida del paso anterior.

El proceso se repite hasta que todos los dispositivos han procesado todos los _embeddings_ de texto.

El resultado final se obtiene con una última operación que suma todas las pérdidas acumuladas en cada dispositivo.